In [1]:
import sys
sys.path.append( '../' )

In [2]:
import numpy as np
import cs_vqe as c
import ast
import os
import scipy as sp
from copy import deepcopy

In [3]:
from openfermion import qubit_operator_sparse
import quchem.Misc_functions.conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [4]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
cwd = os.getcwd()
working_dir = os.path.dirname(cwd)
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [5]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [6]:
# mol_k
ey = 'H2_6-31G_singlet'  
mol_key ='H2-O1_STO-3G_singlet'


# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'IIIIIIIIII': -55.226577416214326,
 'IIIIIIIIIZ': 12.411397933347452,
 'IIIIIIIIYY': -0.007573527729508396,
 'IIIIIIIIZI': 12.411397933347452,
 'IIIIIIIIZZ': 1.1862777224861802,
 'IIIIIIIXIX': -0.10460420805448783,
 'IIIIIIIXXI': 0.0020603107584759067,
 'IIIIIIIXZX': -0.12499935118669338,
 'IIIIIIIYIY': -0.10460420805448783,
 'IIIIIIIYZY': -0.12499935118669338,
 'IIIIIIIZII': 1.6496131029811243,
 'IIIIIIIZIZ': 0.23693603049649825,
 'IIIIIIIZZI': 0.2515869355056082,
 'IIIIIIXIXI': -0.0033258799919286815,
 'IIIIIIXXYY': -0.014650905009109934,
 'IIIIIIXYYX': 0.014650905009109934,
 'IIIIIIXZXI': -0.12499935118669338,
 'IIIIIIXZXZ': -0.10460420805448783,
 'IIIIIIYIYI': -0.0033258799919286815,
 'IIIIIIYXXY': 0.014650905009109934,
 'IIIIIIYYII': -0.025296526569690943,
 'IIIIIIYYXX': -0.014650905009109934,
 'IIIIIIYZYI': -0.12499935118669338,
 'IIIIIIYZYZ': -0.10460420805448783,
 'IIIIIIYZZY': 0.0020603107584759067,
 'IIIIIIZIII': 1.6496131029811243,
 'IIIIIIZIIZ': 0.2515869355056082,
 'IIIII

In [7]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  10


# Get non-contextual H

In [8]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

Split into:

$$H = H_{c} + H_{nc}$$

In [9]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [10]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [11]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [12]:
Z_list

['IIIIIIIIII',
 'IIIIIIIIIZ',
 'IIIIIIIIZI',
 'IIIIIIIIZZ',
 'IIIIIIIZII',
 'IIIIIIIZIZ',
 'IIIIIIIZZI',
 'IIIIIIZIII',
 'IIIIIIZIIZ',
 'IIIIIIZIZI',
 'IIIIIIZZII',
 'IIIIZIIIII',
 'IIIIZIIIIZ',
 'IIIIZIIIZI',
 'IIIIZIIZII',
 'IIIIZIZIII',
 'IIIZIIIIII',
 'IIIZIIIIIZ',
 'IIIZIIIIZI',
 'IIIZIIIZII',
 'IIIZIIZIII',
 'IIIZZIIIII',
 'IIZIIIIIII',
 'IIZIIIIIIZ',
 'IIZIIIIIZI',
 'IIZIIIIZII',
 'IIZIIIZIII',
 'IIZIZIIIII',
 'IIZZIIIIII',
 'IIZZIIZZZZ',
 'IZIIIIIIII',
 'IZIIIIIIIZ',
 'IZIIIIIIZI',
 'IZIIIIIZII',
 'IZIIIIZIII',
 'IZIIZIIIII',
 'IZIZIIIIII',
 'IZIZZIIZIZ',
 'IZZIIIIIII',
 'IZZIZIZIZI',
 'ZIIIIIIIII',
 'ZIIIIIIIIZ',
 'ZIIIIIIIZI',
 'ZIIIIIIZII',
 'ZIIIIIZIII',
 'ZIIIZIIIII',
 'ZIIZIIIIII',
 'ZIIZZIIZIZ',
 'ZIZIIIIIII',
 'ZIZIZIZIZI',
 'ZZIIIIIIII',
 'ZZIIZIIZIZ',
 'ZZIIZIZIZI',
 'ZZIZIIIZIZ',
 'ZZIZZIIIIZ',
 'ZZIZZIIZII',
 'ZZIZZIIZIZ',
 'ZZIZZIIZZZ',
 'ZZIZZIZZIZ',
 'ZZZIIIZIZI',
 'ZZZIZIIIZI',
 'ZZZIZIZIII',
 'ZZZIZIZIZI',
 'ZZZIZIZIZZ',
 'ZZZIZIZZZI',
 'ZZZZZIIZIZ',
 'ZZZZZIZI

In [13]:
T_list

['IIIIIXZZZZ',
 'IIIIIZIIII',
 'IIIIIZIIIZ',
 'IIIIIZIIZI',
 'IIIIIZIZII',
 'IIIIIZZIII',
 'IIIIZXIZZZ',
 'IIIIZXZIZZ',
 'IIIIZXZZIZ',
 'IIIIZXZZZI',
 'IIIIZXZZZZ',
 'IIIIZZIIII',
 'IIIZIZIIII',
 'IIIZZXZZZZ',
 'IIZIIZIIII',
 'IIZIZXZZZZ',
 'IZIIIZIIII',
 'IZIIZXZZZZ',
 'IZIZIZIZIZ',
 'IZIZZXZIZI',
 'IZZIIZZIZI',
 'IZZIZXIZIZ',
 'ZIIIIXIZZZ',
 'ZIIIIXZIZZ',
 'ZIIIIXZZIZ',
 'ZIIIIXZZZI',
 'ZIIIIXZZZZ',
 'ZIIIIZIIII',
 'ZIIIZXZZZZ',
 'ZIIIZZIIII',
 'ZIIIZZIIIZ',
 'ZIIIZZIIZI',
 'ZIIIZZIZII',
 'ZIIIZZZIII',
 'ZIIZIXZZZZ',
 'ZIIZZZIIII',
 'ZIZIIXZZZZ',
 'ZIZIZZIIII',
 'ZZIIIXZZZZ',
 'ZZIIZZIIII',
 'ZZIZIXZIZI',
 'ZZIZZZIZIZ',
 'ZZZIIXIZIZ',
 'ZZZIZZZIZI']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [14]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [15]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['IIIIIIIIII', 'IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIIZZ', 'IIIIIIIZII', 'IIIIIIIZIZ', 'IIIIIIIZZI', 'IIIIIIZIII', 'IIIIIIZIIZ', 'IIIIIIZIZI', 'IIIIIIZZII', 'IIIIZIIIII', 'IIIIZIIIIZ', 'IIIIZIIIZI', 'IIIIZIIZII', 'IIIIZIZIII', 'IIIZIIIIII', 'IIIZIIIIIZ', 'IIIZIIIIZI', 'IIIZIIIZII', 'IIIZIIZIII', 'IIIZZIIIII', 'IIZIIIIIII', 'IIZIIIIIIZ', 'IIZIIIIIZI', 'IIZIIIIZII', 'IIZIIIZIII', 'IIZIZIIIII', 'IIZZIIIIII', 'IIZZIIZZZZ', 'IZIIIIIIII', 'IZIIIIIIIZ', 'IZIIIIIIZI', 'IZIIIIIZII', 'IZIIIIZIII', 'IZIIZIIIII', 'IZIZIIIIII', 'IZIZZIIZIZ', 'IZZIIIIIII', 'IZZIZIZIZI', 'ZIIIIIIIII', 'ZIIIIIIIIZ', 'ZIIIIIIIZI', 'ZIIIIIIZII', 'ZIIIIIZIII', 'ZIIIZIIIII', 'ZIIZIIIIII', 'ZIIZZIIZIZ', 'ZIZIIIIIII', 'ZIZIZIZIZI', 'ZZIIIIIIII', 'ZZIIZIIZIZ', 'ZZIIZIZIZI', 'ZZIZIIIZIZ', 'ZZIZZIIIIZ', 'ZZIZZIIZII', 'ZZIZZIIZIZ', 'ZZIZZIIZZZ', 'ZZIZZIZZIZ', 'ZZZIIIZIZI', 'ZZZIZIIIZI', 'ZZZIZIZIII', 'ZZZIZIZIZI', 'ZZZIZIZIZZ', 'ZZZIZIZZZI', 'ZZZZZIIZIZ', 'ZZZZZIZIZI']
G (independent) Z list: ['ZIIIIIIIII'

In [16]:
print('all Ci1 terms:', Ci1_list)

all Ci1 terms: ['IIIIIZIIII', 'IIIIIXZZZZ']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [17]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'IIIIIIIIII': [[], [], 1],
 'IIIIIIIIIZ': [['IIIIIIIIIZ'], [], 1],
 'IIIIIIIIZI': [['IIIIIIIIZI'], [], 1],
 'IIIIIIIIZZ': [['IIIIIIIIZI', 'IIIIIIIIIZ'], [], 1],
 'IIIIIIIZII': [['IIIIIIIZII'], [], 1],
 'IIIIIIIZIZ': [['IIIIIIIZII', 'IIIIIIIIIZ'], [], 1],
 'IIIIIIIZZI': [['IIIIIIIZII', 'IIIIIIIIZI'], [], 1],
 'IIIIIIZIII': [['IIIIIIZIII'], [], 1],
 'IIIIIIZIIZ': [['IIIIIIZIII', 'IIIIIIIIIZ'], [], 1],
 'IIIIIIZIZI': [['IIIIIIZIII', 'IIIIIIIIZI'], [], 1],
 'IIIIIIZZII': [['IIIIIIZIII', 'IIIIIIIZII'], [], 1],
 'IIIIIXZZZZ': [[], ['IIIIIXZZZZ'], 1],
 'IIIIIZIIII': [[], ['IIIIIZIIII'], 1],
 'IIIIIZIIIZ': [['IIIIIIIIIZ'], ['IIIIIZIIII'], 1],
 'IIIIIZIIZI': [['IIIIIIIIZI'], ['IIIIIZIIII'], 1],
 'IIIIIZIZII': [['IIIIIIIZII'], ['IIIIIZIIII'], 1],
 'IIIIIZZIII': [['IIIIIIZIII'], ['IIIIIZIIII'], 1],
 'IIIIZIIIII': [['IIIIZIIIII'], [], 1],
 'IIIIZIIIIZ': [['IIIIZIIIII', 'IIIIIIIIIZ'], [], 1],
 'IIIIZIIIZI': [['IIIIZIIIII', 'IIIIIIIIZI'], [], 1],
 'IIIIZIIZII': [['IIIIZIIIII', 'IIIIIIIZII'], [], 1]

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [18]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [19]:
fn_form

[9,
 2,
 [[-55.226577416214326, [], [], 'IIIIIIIIII'],
  [12.411397933347452, [8], [], 'IIIIIIIIIZ'],
  [12.411397933347452, [7], [], 'IIIIIIIIZI'],
  [1.1862777224861802, [7, 8], [], 'IIIIIIIIZZ'],
  [1.6496131029811243, [6], [], 'IIIIIIIZII'],
  [0.23693603049649825, [6, 8], [], 'IIIIIIIZIZ'],
  [0.2515869355056082, [6, 7], [], 'IIIIIIIZZI'],
  [1.6496131029811243, [5], [], 'IIIIIIZIII'],
  [0.2515869355056082, [5, 8], [], 'IIIIIIZIIZ'],
  [0.23693603049649825, [5, 7], [], 'IIIIIIZIZI'],
  [0.1817943337373835, [5, 6], [], 'IIIIIIZZII'],
  [-5.791003103036013e-05, [], [1], 'IIIIIXZZZZ'],
  [1.1909223847985508, [], [0], 'IIIIIZIIII'],
  [0.1959701332597126, [8], [0], 'IIIIIZIIIZ'],
  [0.19873710487294718, [7], [0], 'IIIIIZIIZI'],
  [0.12487996093473576, [6], [0], 'IIIIIZIZII'],
  [0.16016822022287117, [5], [0], 'IIIIIZZIII'],
  [1.1909223847985508, [4], [], 'IIIIZIIIII'],
  [0.19873710487294718, [4, 8], [], 'IIIIZIIIIZ'],
  [0.1959701332597126, [4, 7], [], 'IIIIZIIIZI'],
  [0.160168220

In [20]:
Energy_function = c.energy_function(fn_form)

In [21]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [22]:
 Energy_function(*q_variables,*r_variables)

-81.69149175109331

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [23]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-83.87422390061548
[[1, -1, -1, -1, -1, -1, -1, -1, -1], [-0.999999999999984, -1.7850620613845431e-07]]


In [24]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [25]:
ground_state_params

[[1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-0.999999999999984, -1.7850620613845431e-07]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [26]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIIIIIIIII', 'IZIIIIIIII', 'IIZIIIIIII', 'IIIZIIIIII', 'IIIIZIIIII', 'IIIIIIZIII', 'IIIIIIIZII', 'IIIIIIIIZI', 'IIIIIIIIIZ']
['IIIIIZIIII', 'IIIIIXZZZZ']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

In [27]:
H_mat = conv_scr.Get_Openfermion_Hamiltonian(ham)
sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
true_gs = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
true_gs

-83.92870248174636

In [28]:
# # full problem solved
# n_qubits = len(list(nonCon_H.keys())[0])
# output = c.csvqe_approximations_heuristic(ham,
#                                           nonCon_H,
#                                           n_qubits,
#                                           true_gs)


output=[]
output.append([2, 9, 4, 0, 3, 6, 5, 1, 7, 8])


# order of removal!
output[-1]

[2, 9, 4, 0, 3, 6, 5, 1, 7, 8]

In [29]:
quantum_part = set([1,2,3])
all_qubits = set(range(7))
all_qubits.difference(quantum_part)

{0, 4, 5, 6}

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

In [30]:
from cs_vqe_with_SeqRot import diagonalize_epistemic_dictionary_generator

In [31]:
(mapping_GuA_to_singleZ_with_ep_exp_vals, 
 R_sk_OP_list,
GuA_full_rotated) = diagonalize_epistemic_dictionary_generator(model, 
                                                            fn_form, 
                                                            ground_state_params, 
                                                            check_reduction=False)

In [32]:
from cs_vqe_with_SeqRot import get_wills_order
order = deepcopy(output[-1])
print(order)

actual_qubit_removal_order = get_wills_order(GuA_full_rotated, order)

[2, 9, 4, 0, 3, 6, 5, 1, 7, 8]


In [33]:
mapping_GuA_to_singleZ_with_ep_exp_vals

{'ZIIIIIIIII': {'single_Z': 'ZIIIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IZIIIIIIII': {'single_Z': 'IZIIIIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIZIIIIIII': {'single_Z': 'IIZIIIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIZIIIIII': {'single_Z': 'IIIZIIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIZIIIII': {'single_Z': 'IIIIZIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIZIII': {'single_Z': 'IIIIIIZIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIZII': {'single_Z': 'IIIIIIIZII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIIZI': {'single_Z': 'IIIIIIIIZI',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIIIZ': {'single_Z': 'IIIIIIIIIZ',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIZIIII': {'single_Z': 'IIIIIZIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': True}}

In [34]:
from cs_vqe_with_SeqRot import diagonalize_epistemic_SeqRot

In [35]:
# mapping_GuA_to_singleZ_with_ep_exp_vals['ZIIII']['single_Z'].index('Z')

In [36]:
(rotations, 
 do_unitary_part, 
 ep_state_trans,
 GuA) = diagonalize_epistemic_SeqRot(model, mapping_GuA_to_singleZ_with_ep_exp_vals, [3])
GuA

['IIIZIIIIII']

In [37]:
from cs_vqe_with_SeqRot import get_reduced_hamiltonian_by_qubits_fixed

In [38]:
H_reduced = get_reduced_hamiltonian_by_qubits_fixed(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             [3,2,0],
                                             R_sk_OP_list)

In [39]:
from cs_vqe_with_SeqRot import get_reduced_hamiltonians_by_order 

# order_copy = list(range(hamiltonians[mol_key][1]))
order_copy = deepcopy(output[-1])
print(order_copy)
print()

H_reduced_list = get_reduced_hamiltonians_by_order(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             actual_qubit_removal_order,
                                             R_sk_OP_list)
H_reduced_list

[2, 9, 4, 0, 3, 6, 5, 1, 7, 8]



[{'': -83.87422393695655},
 {'I': -83.59030332844156,
  'X': -1.0653572607433048e-09,
  'Z': 0.2839206085149597},
 {'II': -83.17446122858297,
  'IX': 0.05685943367617384,
  'IZ': 0.415842063517476,
  'XI': -0.0013805549974529457,
  'XZ': -0.0013805539320958342,
  'XX': -0.086001709740204,
  'ZI': 0.2809672326667499,
  'ZZ': -0.0029533758482093997,
  'ZX': 0.05685943148609232},
 {'III': -82.62483871771943,
  'IIX': 0.02842971683808694,
  'IIZ': 0.2597586209277342,
  'IXI': 0.01545001100876814,
  'IYY': -0.017238832673645513,
  'IZI': 0.5496225108635344,
  'IZX': -0.02842971683808692,
  'IZZ': -0.15608344258974197,
  'XII': -0.0006902774987264729,
  'XXX': 0.007493741525714776,
  'XIZ': -0.0006902769660479171,
  'XZI': 0.0006902774987264798,
  'XZX': 0.043000854870102,
  'YYX': 0.007493741525714776,
  'ZII': 0.0825619881415175,
  'ZIZ': 0.28910472848934254,
  'ZZI': -0.1984052445252324,
  'ZZX': -0.02842971574304616,
  'ZXX': -0.017238832673645513,
  'IXZ': 0.01545001100876814,
  'XXZ': 

In [40]:
deepcopy(output[-1])

[2, 9, 4, 0, 3, 6, 5, 1, 7, 8]

In [41]:
ww = [0, 1, 3, 4, 6, 7, 8, 9]
for pop_ind in [3, 0, 1, 2, 1, 0, 0]:
    ww.pop(pop_ind)
ww

[9]

In [42]:
# order_copy = list(range(hamiltonians[mol_key][1]))
order_copy = deepcopy(output[-1])

red_Hs = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           order_copy)
red_Hs

set()

{2}

{2, 5}

{2, 4, 5}

{0, 2, 4, 5}

{0, 2, 3, 4, 5}

{0, 2, 3, 4, 5, 7}

{0, 2, 3, 4, 5, 6, 7}

{0, 1, 2, 3, 4, 5, 6, 7}

{0, 1, 2, 3, 4, 5, 6, 7, 8}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}



[{'': -83.87422390061546},
 {'I': -83.59030330225023,
  'X': 1.4286481766698222e-08,
  'Z': 0.2839205983651979},
 {'II': -83.17446122858297,
  'IX': -0.05685935944578392,
  'IZ': -0.41584207366723114,
  'XI': -0.0013805549974529457,
  'XZ': 0.0013805692839347367,
  'XX': 0.08600170949376519,
  'ZI': 0.2809672326667499,
  'ZZ': 0.002953365698447924,
  'ZX': -0.05685943201328733},
 {'III': -82.62483871771943,
  'IIX': -0.02842967046956053,
  'IIZ': -0.25975862600261085,
  'IXI': 0.01545001100876814,
  'IYY': -0.017238832673645513,
  'IZI': 0.5496225108635344,
  'IZX': 0.02842968897622332,
  'IZZ': 0.15608344766462037,
  'XII': -0.0006902774987264729,
  'XXX': -0.007493739248790949,
  'XXZ': -0.012755432967272436,
  'XIZ': 0.0006902846419673686,
  'XIX': 0.04300085474688259,
  'XZI': 0.0006902774987264798,
  'XZX': -0.04300085474688259,
  'XZZ': -0.0006902846419673692,
  'YYX': -0.007493739248790949,
  'YYZ': -0.012755432967272436,
  'ZII': 0.0825619881415175,
  'ZIZ': -0.2891047335642187

In [43]:
len(H_reduced_list) == len(red_Hs)

True

In [45]:
E_list_one =[]
for H in red_Hs:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    print(E)
    print(len(H))
    print()
    E_list_one.append(E)

-83.87422390061546
1

-83.87422390061545
3

-83.87959838666477
9

-83.88315847347786
34

-83.89785357843976
73

-83.91205890664989
136

-83.91813133139065
277

-83.92761706906066
449

-83.92862840710193
534

-83.9286535706245
766

-83.92870251134427
1067



In [46]:
E_list_two =[]
for H in H_reduced_list:
#     H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    print(E)
    print(len(H))
    print()
    E_list_two.append(E)

-83.87422393695655
1

-83.87422393695655
3

-83.87959838666478
9

-83.8831584734777
34

-83.89785356786022
71

-83.91205889216413
132

-83.91813131301484
267

-83.92761703951213
435

-83.92862837750984
514

-83.92865354102884
740

-83.92870248174599
1035



In [47]:
np.array(E_list_two) - np.array(E_list_one)

array([-3.63410919e-08, -3.63411061e-08, -1.42108547e-14,  1.56319402e-13,
        1.05795408e-08,  1.44857637e-08,  1.83758146e-08,  2.95485307e-08,
        2.95920870e-08,  2.95956681e-08,  2.95982829e-08])

In [48]:
# cs_VQE new
np.array(E_list_two) - true_gs

array([5.44785448e-02, 5.44785448e-02, 4.91040951e-02, 4.55440083e-02,
       3.08489139e-02, 1.66435896e-02, 1.05711687e-02, 1.08544223e-03,
       7.41042365e-05, 4.89407175e-05, 3.69482223e-13])

In [49]:
# cs_VQE will
np.array(E_list_one) - true_gs

array([ 5.44785811e-02,  5.44785811e-02,  4.91040951e-02,  4.55440083e-02,
        3.08489033e-02,  1.66435751e-02,  1.05711504e-02,  1.08541269e-03,
        7.40746444e-05,  4.89111219e-05, -2.95979135e-08])

In [50]:
order_copy = deepcopy(output[-1])
order_copy

[2, 9, 4, 0, 3, 6, 5, 1, 7, 8]

In [51]:
# look for: do_unitary_part=TRUE
mapping_GuA_to_singleZ_with_ep_exp_vals


{'ZIIIIIIIII': {'single_Z': 'ZIIIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IZIIIIIIII': {'single_Z': 'IZIIIIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIZIIIIIII': {'single_Z': 'IIZIIIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIZIIIIII': {'single_Z': 'IIIZIIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIZIIIII': {'single_Z': 'IIIIZIIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIZIII': {'single_Z': 'IIIIIIZIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIZII': {'single_Z': 'IIIIIIIZII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIIZI': {'single_Z': 'IIIIIIIIZI',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIIIIZ': {'single_Z': 'IIIIIIIIIZ',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIZIIII': {'single_Z': 'IIIIIZIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': True}}

In [52]:
'IIIIIZIIII'.index('Z')

5